### Original data

In [1]:
from pathlib import Path

import requests
from tqdm import tqdm

import pandas as pd

In [2]:
ORIG_DATA_PATH = Path('./orig_data')

if not ORIG_DATA_PATH.exists():
    ORIG_DATA_PATH.mkdir(exist_ok=True)

urls = ['https://datasets.figure-eight.com/figure_eight_datasets/disaster_response_data/disaster_response_messages_training.csv',
        'https://datasets.figure-eight.com/figure_eight_datasets/disaster_response_data/disaster_response_messages_test.csv',
        'https://datasets.figure-eight.com/figure_eight_datasets/disaster_response_data/disaster_response_messages_validation.csv']

file_paths = [ORIG_DATA_PATH/'train_df.csv', 
              ORIG_DATA_PATH/'test_df.csv',
              ORIG_DATA_PATH/'valid_df.csv']

def download_file(url, file_path):
    r = requests.get(url)

    with open(file_path, 'w') as f:
        f.write(r.text[3:])
        
for url, file_path in tqdm(zip(urls, file_paths)):
    download_file(url, file_path)

3it [00:01,  1.87it/s]


In [3]:
dfs = []

for file_path in tqdm(file_paths):
    dfs.append(pd.read_csv(file_path))
    
full_df = pd.concat(dfs, axis=0)
full_df = full_df.drop('PII', axis=1)
print(full_df.shape)
full_df.head()

  0%|          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100%|██████████| 3/3 [00:00<00:00, 13.77it/s]

(26248, 41)


,id,split,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,train,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,train,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,12,train,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14,train,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,train,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
full_df.loc[:, 'related':'direct_report'].apply(pd.value_counts).T

,0,1,2
related,6125.0,19930.0,193.0
request,21768.0,4480.0,NaN
offer,26129.0,119.0,NaN
aid_related,15370.0,10878.0,NaN
medical_help,24161.0,2087.0,NaN
medical_products,24934.0,1314.0,NaN
search_and_rescue,25524.0,724.0,NaN
security,25777.0,471.0,NaN
military,25388.0,860.0,NaN
child_alone,26248.0,NaN,NaN


# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [106]:
# import libraries
from sqlalchemy import create_engine

import numpy as np

In [107]:
# load messages datase
messages = pd.read_csv('./messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [108]:
messages[messages.duplicated(keep=False)]

,id,message,original,genre
162,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct
163,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct
655,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct
656,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct
709,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct
710,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct
1407,1652,"please we need water, food and tents, we have ...","p jwen dlo, manje, tant pou nou demi nou gen 1...",direct
1408,1652,"please we need water, food and tents, we have ...","p jwen dlo, manje, tant pou nou demi nou gen 1...",direct
2130,2446,How much money did TV Latino American collect ...,Konbyen kob tv latino america ranmase miami,direct
2131,2446,How much money did TV Latino American collect ...,Konbyen kob tv latino america ranmase miami,direct


In [109]:
# load categories dataset
categories = pd.read_csv('./categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [110]:
categories[categories.duplicated(keep=False)]

,id,categories
655,804,related-2;request-0;offer-0;aid_related-0;medi...
656,804,related-2;request-0;offer-0;aid_related-0;medi...
2130,2446,related-1;request-0;offer-0;aid_related-0;medi...
2131,2446,related-1;request-0;offer-0;aid_related-0;medi...
2792,3217,related-1;request-1;offer-0;aid_related-1;medi...
2793,3217,related-1;request-1;offer-0;aid_related-1;medi...
3851,4412,related-1;request-1;offer-0;aid_related-1;medi...
3852,4412,related-1;request-1;offer-0;aid_related-1;medi...
4629,5265,related-2;request-0;offer-0;aid_related-0;medi...
4630,5265,related-2;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [111]:
# merge datasets
df = messages.merge(categories, how='inner', on='id')
print(df.shape)
df.head()

(26386, 5)


,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [112]:
len(df[df.duplicated(keep=False)])

273

In [113]:
messages.drop_duplicates().shape, categories.drop_duplicates().shape, df.drop_duplicates().shape

((26180, 4), (26216, 2), (26216, 5))

In [114]:
messages.shape, categories.shape, df.shape

((26248, 4), (26248, 2), (26386, 5))

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [115]:
parsed_categories = df['categories'].str.split(';|-')
parsed_categories_df = parsed_categories.str[1::2].apply(pd.Series)
parsed_categories_df = parsed_categories_df.astype('int8')
parsed_categories_df.columns = parsed_categories.str[0::2].iloc[0]
parsed_categories_df

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [116]:
parsed_categories_df.apply(pd.value_counts).T

,0,1,2
related,6140.0,20042.0,204.0
request,21873.0,4513.0,NaN
offer,26265.0,121.0,NaN
aid_related,15432.0,10954.0,NaN
medical_help,24287.0,2099.0,NaN
medical_products,25067.0,1319.0,NaN
search_and_rescue,25661.0,725.0,NaN
security,25915.0,471.0,NaN
military,25523.0,863.0,NaN
child_alone,26386.0,NaN,NaN


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [117]:
# drop the original categories column from `df`
df = df.drop('categories', axis=1)
df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [118]:
# concatenate the original dataframe with the new `categories` dataframe
print(df.shape, parsed_categories_df.shape)

df = pd.concat([df, parsed_categories_df], axis=1)
print(df.shape)

df.head()

(26386, 4) (26386, 36)
(26386, 40)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [119]:
# check number of duplicates
df.shape, df.duplicated().sum()

((26386, 40), 170)

In [120]:
# drop duplicates
df = df.drop_duplicates()
df.shape

(26216, 40)

In [121]:
# check number of duplicates
df.shape, df.duplicated().sum()

((26216, 40), 0)

### Select data

In [122]:
df[df['related'].isin([0, 2])].loc[:, 'request':].sum(axis=1).value_counts()

0    6310
dtype: int64

In [123]:
df[df['related'] == 1].loc[:, 'request':].sum(axis=1).value_counts()

0     5121
2     3609
4     2956
3     2276
5     1903
6     1347
7      880
8      577
1      443
9      344
10     183
11     107
12      57
13      38
14      20
16      12
15       8
17       7
18       5
19       4
20       4
26       2
21       2
23       1
dtype: int64

In [134]:
related_map = {0: 0, 1: 1, 2: 0}

df['related'] = df['related'].map(related_map).astype('int8')
df.loc[:, 'related':].apply(pd.value_counts).T

,0,1
related,6310.0,19906.0
request,21742.0,4474.0
offer,26098.0,118.0
aid_related,15356.0,10860.0
medical_help,24132.0,2084.0
medical_products,24903.0,1313.0
search_and_rescue,25492.0,724.0
security,25745.0,471.0
military,25356.0,860.0
child_alone,26216.0,NaN


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [135]:
engine = create_engine('sqlite:///DisasterResponse.db')
df.to_sql('DisasterData', engine, if_exists='replace', index=False)

In [136]:
df.shape, pd.read_sql_table('DisasterData', engine).shape

((26216, 40), (26216, 40))

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.